In [1]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
from sklearn.svm import LinearSVC, SVC
import os
import warnings
from sklearn.exceptions import ConvergenceWarning
import scipy.misc

In [2]:
path = '../datasets/imagenes_segmentacion'
img_path  = '/images/'
mark_path = '/bordes_mr/'
real_path = '/bordes_real/' 
file_names = os.listdir(path+img_path)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

    acc_linear, acc_svc = 0,0
    n,m = 0,0

    for idx, filename in enumerate(file_names):

        # if idx > 2: break

        img = cv2.imread(path+img_path+filename)
        mark = cv2.imread(path+mark_path+filename)
        real = cv2.imread(path+real_path+filename)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        real = cv2.cvtColor(real, cv2.COLOR_BGR2GRAY)
        mark = cv2.GaussianBlur(mark,(5,5),0)

        nrows,ncols,nch = img.shape
        # Image vectorization
        Ximg = np.reshape(img,(nrows*ncols,nch))
        Xmask = np.reshape(mark,(nrows*ncols,nch))[:,0]

        values = np.unique(Xmask)

        Xobj = Ximg[ Xmask==255, : ]
        Xbck = Ximg[ Xmask==64, :]
        Xtrain = np.concatenate((Xobj,Xbck),axis=0)

        Yobj = np.ones((len(Xobj)))*255
        Ybck = np.zeros((len(Xbck)))
        Ytrain = np.concatenate((Yobj,Ybck),axis=0)

        modelL = LinearSVC()
        modelL.fit(Xtrain,Ytrain)
        XpredL = modelL.predict(Ximg)
        predL = np.reshape(XpredL,(nrows,ncols))
        
        #predL = cv2.merge((predL,predL,predL))
        #predL = cv2.cvtColor(predL.astype('uint8'), cv2.COLOR_GRAY2BGR)
        #predL = cv2.fastNlMeansDenoisingColored(predL,None,10,10,7,21)
        #predL = cv2.cvtColor(predL, cv2.COLOR_RGB2GRAY)
        
        model = SVC()
        model.fit(Xtrain,Ytrain)
        Xpred = model.predict(Ximg)
        pred = np.reshape(Xpred,(nrows,ncols))

        #holes = cv2.floodFill(predL.copy(), mask, (0, 0), 255)[1]
        #holes = ~holes
        #predL[holes == 255] = 255

        acc_linear += np.sum( predL.astype(int) == real )
        acc_svc    += np.sum( predL.astype(int) == real )

        n+=real.size
        m+=real.size
        
        #plt.figure(figsize=(10,10))
        #plt.subplot(2,2,1)
        #plt.imshow(img)
        #plt.subplot(2,2,2)
        #plt.imshow(real,cmap='gray')
        #plt.subplot(2,2,3)
        #plt.title('LinearSVC')
        #plt.imshow(predL,cmap='gray')
        #plt.subplot(2,2,4)
        #plt.title('SVC')
        #plt.imshow(pred,cmap='gray')
        #plt.show()


In [4]:
print('Accuracy LinearSVC ', acc_linear/n)
print('Accuracy SVC ', acc_svc/m)

Accuracy LinearSVC  0.8689925347222223
Accuracy SVC  0.8689925347222223
